In [1]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
!pip install watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.8 MB/s eta 0:00:00


In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00


In [5]:
import os
import os.path as op
import time
import os
import sys
import tarfile
import time
import numpy as np
import pandas as pd
import torch
import torchmetrics
import urllib
import os
import csv
import pandas as pd

from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import BartForSequenceClassification, BartTokenizer
from watermark import watermark
from packaging import version
from torch.utils.data import Dataset
from tqdm import tqdm
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import StratifiedShuffleSplit
from datasets import load_dataset, Features, Value, ClassLabel

In [6]:
from datasets import load_dataset

# Load MNLI dataset
mnli_dataset = load_dataset("multi_nli")
mnli_train_small = mnli_dataset['train'][0:15000]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

In [7]:
mnli_small=pd.DataFrame([])
mnli_small['premise']=mnli_train_small['premise']
mnli_small['hypothesis']=mnli_train_small['hypothesis']
mnli_small['class']=mnli_train_small['label']
mnli_small.to_csv("Multi_NLI.csv", index=False, encoding="utf-8")

Fine-tuning Code based on: https://github.com/levalencia/DataScience-Portfolio/blob/main/FineTuningDistilbert/HateSpeechFineTuningWithDistilberta.ipynb?source=post_page-----69f944189648--------------------------------

In [8]:
import time
import sys
def reporthook(count, block_size, total_size):
    global start_time

    # If this is the first block being downloaded
    if count == 0:
        # Record the current time
        start_time = time.time()
        return

    # Calculate the elapsed time since the start of the download
    duration = time.time() - start_time

    # Calculate the size of the downloaded block in bytes
    progress_size = int(count * block_size)

    # Calculate the download speed in MB/s
    speed = progress_size / (1024.0**2 * duration)

    # Calculate the percentage of the download completed
    percent = count * block_size * 100.0 / total_size

    # Write the progress report to stdout and flush the buffer
    sys.stdout.write(
        f"\r{int(percent)}% | {progress_size / (1024.**2):.2f} MB "
        f"| {speed:.2f} MB/s | {duration:.2f} sec elapsed"
    )
    sys.stdout.flush()

In [9]:
import os
import csv

def download_dataset():
    # Set the file name to download
    data_file = "/content/Multi_NLI.csv"

    # Check if the file exists in the current working directory
    if not os.path.isfile(data_file):
        # If the file does not exist, print an error message and return None
        print(f"Error: {data_file} not found")
        return None

    # Open the file in read mode
    with open(data_file, 'r') as f:
        # Use csv.DictReader to read the file and create dictionaries for each row
        reader = csv.DictReader(f)
        # Initialize an empty list to store the data
        data=[]
        # Loop through each row in the file
        for row in reader:
            # Extract the class and tweet values from the row
            class_ = row['class']
            tweet = row['tweet']
            # Create a tuple with the extracted values and append it to the data list
            data.append((class_, tweet))

    # Return the data list
    return data

In [10]:
def load_dataset_into_dataframe(): # define function
    data_file = "/content/Multi_NLI.csv" # assign file name to a variable

    try: # try to read the file
        df = pd.read_csv(data_file, on_bad_lines='skip') # read csv file with pandas
    except FileNotFoundError: # if file is not found
        print(f"Error: {data_file} not found") # print error message

    df["class"] = df["class"].astype(int) # convert "class" column to integer type
    df = df.sample(frac=1, random_state=42).reset_index(drop=True) # shuffle the rows randomly
    #df.drop("Unnamed: 0", inplace=True, axis=1) # remove "Unnamed: 0" column
    print("Class distribution:") # print message
    print(df["class"].value_counts()) # print the count of each class
    return df # return the dataframe

In [11]:
def partition_dataset(df): # define function that takes a pandas dataframe as input
    df_shuffled = df.sample(frac=1, random_state=1).reset_index(drop=True) # shuffle the rows of the dataframe randomly

    num_rows = len(df_shuffled) # calculate the number of rows in the shuffled dataframe
    num_train = int(num_rows * 0.7) # calculate the number of rows to include in the training set (70% of total rows)
    num_val = int(num_rows * 0.15) # calculate the number of rows to include in the validation set (15% of total rows)

    df_train = df_shuffled.iloc[:num_train] # slice the first num_train rows and assign them to the training set
    df_val = df_shuffled.iloc[num_train:num_train+num_val] # slice the next num_val rows and assign them to the validation set
    df_test = df_shuffled.iloc[num_train+num_val:] # slice the remaining rows and assign them to the test set

    df_train.to_csv("train.csv", index=False, encoding="utf-8") # write the training set to a CSV file named "train.csv"

    df_val.to_csv("val.csv", index=False, encoding="utf-8") # write the validation set to a CSV file named "val.csv"
    df_test.to_csv("test.csv", index=False, encoding="utf-8") # write the test set to a CSV file named "test.csv"

In [12]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        """
        Reads data from a CSV file and stores it in the dataset object.
        """
        self.data = []
        self.labels = []  # Store labels separately
        with open(file_path, 'r') as f:
            reader = csv.DictReader(f)
            for row in reader:
                class_ = row['class']  # Extract the class label from the row
                premise = row['premise']  # Extract the tweet from the row
                hypothesis = row['hypothesis']
                self.data.append((class_, premise, hypothesis))  # Store the tweet and class label as a tuple
                self.labels.append(class_)  # Store the label in a separate list

    def __len__(self):
        """
        Returns the length of the dataset.
        """
        return len(self.data)

    def __getitem__(self, index):
        """
        Retrieves a single item (a tweet and its corresponding class label) from the dataset at the specified index.
        Tokenizes the tweet using the tokenizer object and returns the input IDs, attention mask, and label as a tuple.
        """
        class_, premise, hypothesis = self.data[index]  # Retrieve the tweet and class label at the specified index
        encoding = tokenizer.encode_plus(premise, hypothesis, add_special_tokens=True, max_length=256, padding='max_length', return_tensors='pt', truncation=True)  # Tokenize the tweet using the tokenizer object
        input_ids = encoding['input_ids'][0]  # Get the input IDs from the tokenization output
        attention_mask = encoding['attention_mask'][0]  # Get the attention mask from the tokenization output
        label = int(self.labels[index])  # Retrieve the label for the tweet and convert it to an integer
        return input_ids, attention_mask, label  # Return the input IDs, attention mask, and label as a tuple

In [13]:
def tokenize_text(batch):
    # expects a dictionary `batch` with a key `"text"` containing a string of text to be tokenized
    return tokenizer(batch['premise'], batch['hypothesis'], truncation=True, padding=True)
    # tokenize the text in `batch["text"]` using the `tokenizer` object
    # `truncation=True` tells the tokenizer to truncate the sequence of tokens if it exceeds a certain length
    # `padding=True` tells the tokenizer to pad the sequence of tokens with special tokens (e.g., `[PAD]`) to ensure that all inputs have the same length
    # the function returns the tokenized text as a list of integers, where each integer corresponds to a particular token in the vocabulary of the `tokenizer`

In [14]:
def train(num_epochs, model, optimizer, train_loader, val_loader, device):
    # trains a neural network model for a given number of epochs on a given dataset
    # `num_epochs`: number of epochs to train for
    # `model`: neural network model to train
    # `optimizer`: optimizer algorithm to use during training (e.g. Adam or SGD)
    # `train_loader`: data loader for training set
    # `val_loader`: data loader for validation set
    # `device`: device to use for computation (e.g. "cpu" or "cuda")

    for epoch in range(num_epochs):
        # iterate over each epoch

        # initialize a new instance of the Accuracy metric for training set
        train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=3).to(device)
        for batch_idx,batch in enumerate(train_loader):
            model.train()
            input_ids, attention_mask, label = batch  # unpack the tuple into its elements
            # move the tensors to the device
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            label = label.to(device)
            # assign the tensors to the batch dictionary
            batch = {"input_ids": input_ids, "attention_mask": attention_mask, "label": label}

            ### FORWARD AND BACK PROP
            outputs = model(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                labels=batch["label"],
            )

            optimizer.zero_grad()
            outputs["loss"].backward()

            ### UPDATE MODEL PARAMETERS
            optimizer.step()

            ### LOGGING
            #if not batch_idx % 300:
            print(f"Epoch: {epoch+1:04d}/{num_epochs:04d} | Batch {batch_idx:04d}/{len(train_loader):04d} | Loss: {outputs['loss']:.4f}")

            model.eval()
            with torch.no_grad():
                predicted_labels = torch.argmax(outputs["logits"], 1)
                train_acc.update(predicted_labels, batch["label"])

        ### MORE LOGGING
        with torch.no_grad():
            model.eval()
            val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=3).to(device)
            for batch in val_loader:
                input_ids, attention_mask, label = batch  # unpack the tuple into its elements
                # move the tensors to the device
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                label = label.to(device)
                # assign the tensors to the batch dictionary
                batch = {"input_ids": input_ids, "attention_mask": attention_mask, "label": label}

                outputs = model(
                    batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                    labels=batch["label"],
                )
                predicted_labels = torch.argmax(outputs["logits"], 1)
                val_acc.update(predicted_labels, batch["label"])

            print(
                f"Epoch: {epoch+1:04d}/{num_epochs:04d} | Train acc.: {train_acc.compute()*100:.2f}% | Val acc.: {val_acc.compute()*100:.2f}%"
            )

In [15]:
if __name__ == "__main__":
    print(watermark(packages="torch,lightning,transformers", python=True))
    print("Torch CUDA available?", torch.cuda.is_available())
    device = "cuda:0" if torch.cuda.is_available() else "cpu"

    torch.manual_seed(123)
    ##########################
    ### 1 Loading the Dataset
    ##########################
    #download_dataset()
    df = load_dataset_into_dataframe()
    if not (op.exists("train.csv") and op.exists("val.csv") and op.exists("test.csv")):
        partition_dataset(df)

    features = Features(
        {
            "premise": Value(dtype="string"),
            "hypothesis": Value(dtype="string"),
            "class": ClassLabel(
                num_classes=3,
                names=[0, 1, 2],
            )
        })

    mnli_dataset = load_dataset(
        "csv",
        data_files={
            "train": "train.csv",
            "validation": "val.csv",
            "test": "test.csv",
        },
        features=features
    )

    #########################################
    ### 2 Tokenization and Numericalization
    #########################################

    tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
    print("Tokenizer input max length:", tokenizer.model_max_length, flush=True)
    print("Tokenizer vocabulary size:", tokenizer.vocab_size, flush=True)
    print("Tokenizing ...", flush=True)
    mnli_tokenized = mnli_dataset.map(tokenize_text, batched=True, batch_size=None)
    del mnli_dataset
    mnli_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "premise", "hypothesis"])
    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    #########################################
    ### 3 Set Up DataLoaders
    #########################################

    # Define split ratios and other parameters
    train_split = 0.7
    validation_split = 0.15
    test_split = 0.15
    shuffle_dataset = True
    random_seed = 42
    batch_size = 16

    # Create dataset object
    dataset = CustomDataset("/content/Multi_NLI.csv")

    # Implement stratified splits
    splitter_train = StratifiedShuffleSplit(n_splits=1, test_size=1 - train_split, random_state=random_seed)
    train_indices, remaining_indices = next(splitter_train.split(np.zeros(len(dataset)), dataset.labels))

    splitter_test_val = StratifiedShuffleSplit(n_splits=1, test_size=test_split/(test_split+validation_split), random_state=random_seed)
    test_indices, val_indices = next(splitter_test_val.split(np.zeros(len(remaining_indices)), [dataset.labels[i] for i in remaining_indices]))

    # Create PT data samplers and loaders
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)
    test_sampler = SubsetRandomSampler(test_indices)

    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    validation_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)


    #########################################
    ### 4 Initializing the Model
    #########################################

    model = BartForSequenceClassification.from_pretrained(
        "facebook/bart-large", num_labels=3
    )

    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

    #########################################
    ### 5 Finetuning
    #########################################

    start = time.time()
    train(
        num_epochs=3,
        model=model,
        optimizer=optimizer,
        train_loader=train_loader,
        val_loader=validation_loader,
        device=device,
    )

    end = time.time()
    elapsed = end - start
    print(f"Time elapsed {elapsed/60:.2f} min")

    with torch.no_grad():
        model.eval()
        test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=3).to(device)
        for batch in test_loader:
            input_ids, attention_mask, label = batch  # unpack the tuple into its elements
            # move the tensors to the device
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            label = label.to(device)
            # assign the tensors to the batch dictionary
            batch = {"input_ids": input_ids, "attention_mask": attention_mask, "label": label}

            outputs = model(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                labels=batch["label"],
            )
            predicted_labels = torch.argmax(outputs["logits"], 1)
            test_acc.update(predicted_labels, batch["label"])

    print(f"Test accuracy {test_acc.compute()*100:.2f}%")

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

torch       : 2.2.1+cu121
lightning   : not installed
transformers: 4.38.2

Torch CUDA available? True
Class distribution:
2    5504
0    5099
1    4397
Name: class, dtype: int64


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

Epoch: 0001/0003 | Batch 0470/0657 | Loss: 1.1007
Epoch: 0001/0003 | Batch 0471/0657 | Loss: 1.1015
Epoch: 0001/0003 | Batch 0472/0657 | Loss: 1.1064
Epoch: 0001/0003 | Batch 0473/0657 | Loss: 1.1085
Epoch: 0001/0003 | Batch 0474/0657 | Loss: 1.0419
Epoch: 0001/0003 | Batch 0475/0657 | Loss: 1.0941
Epoch: 0001/0003 | Batch 0476/0657 | Loss: 1.0951
Epoch: 0001/0003 | Batch 0477/0657 | Loss: 1.0876
Epoch: 0001/0003 | Batch 0478/0657 | Loss: 1.0777
Epoch: 0001/0003 | Batch 0479/0657 | Loss: 1.1345
Epoch: 0001/0003 | Batch 0480/0657 | Loss: 1.1582
Epoch: 0001/0003 | Batch 0481/0657 | Loss: 1.0926
Epoch: 0001/0003 | Batch 0482/0657 | Loss: 1.1193
Epoch: 0001/0003 | Batch 0483/0657 | Loss: 1.0870
Epoch: 0001/0003 | Batch 0484/0657 | Loss: 1.0860
Epoch: 0001/0003 | Batch 0485/0657 | Loss: 1.0971
Epoch: 0001/0003 | Batch 0486/0657 | Loss: 1.0954
Epoch: 0001/0003 | Batch 0487/0657 | Loss: 1.0529
Epoch: 0001/0003 | Batch 0488/0657 | Loss: 1.1025
Epoch: 0001/0003 | Batch 0489/0657 | Loss: 1.1074


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Epoch: 0002/0003 | Batch 0073/0657 | Loss: 1.0844
Epoch: 0002/0003 | Batch 0074/0657 | Loss: 1.0384
Epoch: 0002/0003 | Batch 0075/0657 | Loss: 1.0645
Epoch: 0002/0003 | Batch 0076/0657 | Loss: 1.1744
Epoch: 0002/0003 | Batch 0077/0657 | Loss: 1.1297
Epoch: 0002/0003 | Batch 0078/0657 | Loss: 1.0499
Epoch: 0002/0003 | Batch 0079/0657 | Loss: 1.0749
Epoch: 0002/0003 | Batch 0080/0657 | Loss: 1.1210
Epoch: 0002/0003 | Batch 0081/0657 | Loss: 1.1596
Epoch: 0002/0003 | Batch 0082/0657 | Loss: 1.1867
Epoch: 0002/0003 | Batch 0083/0657 | Loss: 1.0670
Epoch: 0002/0003 | Batch 0084/0657 | Loss: 1.1007
Epoch: 0002/0003 | Batch 0085/0657 | Loss: 1.1016
Epoch: 0002/0003 | Batch 0086/0657 | Loss: 1.1211
Epoch: 0002/0003 | Batch 0087/0657 | Loss: 1.0881
Epoch: 0002/0003 | Batch 0088/0657 | Loss: 1.0305
Epoch: 0002/0003 | Batch 0089/0657 | Loss: 1.0935
Epoch: 0002/0003 | Batch 0090/0657 | Loss: 1.0829
Epoch: 0002/0003 | Batch 0091/0657 | Loss: 1.1534
Epoch: 0002/0003 | Batch 0092/0657 | Loss: 1.0880


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Epoch: 0003/0003 | Batch 0476/0657 | Loss: 1.0944
Epoch: 0003/0003 | Batch 0477/0657 | Loss: 1.0896
Epoch: 0003/0003 | Batch 0478/0657 | Loss: 1.1032
Epoch: 0003/0003 | Batch 0479/0657 | Loss: 1.1018
Epoch: 0003/0003 | Batch 0480/0657 | Loss: 1.0899
Epoch: 0003/0003 | Batch 0481/0657 | Loss: 1.1009
Epoch: 0003/0003 | Batch 0482/0657 | Loss: 1.1143
Epoch: 0003/0003 | Batch 0483/0657 | Loss: 1.0931
Epoch: 0003/0003 | Batch 0484/0657 | Loss: 1.1475
Epoch: 0003/0003 | Batch 0485/0657 | Loss: 1.0413
Epoch: 0003/0003 | Batch 0486/0657 | Loss: 1.1084
Epoch: 0003/0003 | Batch 0487/0657 | Loss: 1.0725
Epoch: 0003/0003 | Batch 0488/0657 | Loss: 1.1480
Epoch: 0003/0003 | Batch 0489/0657 | Loss: 1.0815
Epoch: 0003/0003 | Batch 0490/0657 | Loss: 1.1053
Epoch: 0003/0003 | Batch 0491/0657 | Loss: 1.0785
Epoch: 0003/0003 | Batch 0492/0657 | Loss: 1.0859
Epoch: 0003/0003 | Batch 0493/0657 | Loss: 1.1044
Epoch: 0003/0003 | Batch 0494/0657 | Loss: 1.1311
Epoch: 0003/0003 | Batch 0495/0657 | Loss: 1.1580


In [16]:
from pathlib import Path

# 1. Create models directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok= True)

# 2. Create model save path
MODEL_NAME = "bartfinetuned.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(),
           f=MODEL_SAVE_PATH)

Saving model to: models/bartfinetuned.pth


In [19]:
import shutil

# Mount Google Drive to Colab
from google.colab import drive
drive.mount('/content/drive')

# Define the source file path (in Colab)
source_file_path = '/content/models/bartfinetuned.pth'

# Define the destination folder path (in Google Drive)
destination_folder_path = '/content/drive/MyDrive/BART Fintuned MNLI/First 15000'

# Move the file to the destination folder
shutil.move(source_file_path, destination_folder_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/BART Fintuned MNLI/First 15000/bartfinetuned.pth'